In [5]:
!pip install transformers tokenizers datasets

from datasets import load_dataset
from tokenizers import ByteLevelBPETokenizer
from transformers import RobertaTokenizerFast

In [6]:
# Tải tập dữ liệu
dataset = load_dataset("PaulTran/vietnamese_spelling_error_detection")

# Trích xuất các văn bản từ cột 'input_text' và 'target_text'
texts = dataset['train']['input_text'] + dataset['train']['target_text']

# Lưu văn bản vào tệp tạm thời để huấn luyện tokenizer
with open("temp_texts.txt", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(text + "\n")
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=["temp_texts.txt"], vocab_size=30_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])


In [8]:
# Lưu tokenizer
import os

# Create the directory if it doesn't exist
os.makedirs("tokenizer", exist_ok=True)

try:
  tokenizer.save_model("tokenizer")
  print("Tokenizer saved successfully!")
except OSError as e:
  print(f"Error saving tokenizer: {e}")

# Tạo RobertaTokenizerFast từ tokenizer đã huấn luyện
tokenizer = RobertaTokenizerFast.from_pretrained("tokenizer")

AttributeError: 'RobertaTokenizerFast' object has no attribute 'save_model'

In [10]:
# Kiểm tra tokenizer mới
text = dataset['train']['input_text'][10]
encoded_input = tokenizer.encode(text)
print("Mã hóa văn bản:")
print(len(encoded_input))
print(encoded_input)
decoded_text = tokenizer.decode(encoded_input)
print("\nGiải mã văn bản:")
print(decoded_text)
print(len(decoded_text.split(' ')))

Mã hóa văn bản:
47
[0, 1607, 1235, 1656, 2719, 10211, 861, 705, 432, 625, 1657, 1077, 776, 441, 1066, 1656, 787, 476, 497, 764, 16, 1103, 476, 7059, 1087, 364, 11777, 17, 11672, 3243, 9976, 17, 1460, 16, 4493, 17, 12045, 16, 10119, 421, 303, 1764, 494, 7177, 757, 1856, 2]

Giải mã văn bản:
<s>Các cầu thủ U23 Việt Nam làm nên kỳ tích trước các đối thủ lớn về thể hình, mạnh về tốc độ như Uzơ-bê–kit- tan, I-rắc, Quata cũng là nhờ vàoqa bản lĩnh</s>
33
